In [1]:
import glob
import os
import pathlib
from pathlib import Path

import pandas as pd

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
os.getcwd()

'/home/wangyh/uro_biomarker/patho_AI'

In [3]:
df = pd.read_csv("./config/dir_uuid.csv") #用于pretraining的uuid
uuid = df["dir_uuid"]  # 获取符合条件的uuid

slides = glob.glob(f"/mnt/wangyh/TCGA_bladder/TCGA_bladder/*/*.svs") #所有slide的path
wsi_path = []
for i in slides:
    id = os.path.split(Path(i).parent)[1]
    if id in uuid.values:
        wsi_path.append(i)

# print(wsi_path)
# print(len(wsi_path))
pd.Series(wsi_path).to_csv('./config/wsi_path_for_pretraining.csv')

# 获取cart下载的压缩包中的数据，其中有clinical.xml及omf.xml

In [ ]:
clin_data = glob.glob('/home/wangyh/uro_biomarker/patho_AI/clinical_data_files/*/*.xml')
clin_data

In [5]:
len(clin_data)

573

# 获取clinical的数据

In [ ]:
clinical_data = pd.read_table('./clinical_cart/clinical.tsv',sep='\t')
clinical_data.columns.values

In [7]:
df = pd.read_csv('./config/TCGA TMB.csv')
TMB = df[df['Cohort']=='BLCA']

# 合并数据，输出到csv，用R看一下

In [8]:
clinical_data.merge(TMB,left_on='case_submitter_id',right_on='ID',how='inner').to_csv('./config/merged_clinical_data_with_TMB.csv')

# 用python看一下csv，提取有用的列进行下一步处理

In [9]:
merged = pd.read_csv('./config/merged_clinical_data_with_TMB.csv')[['case_submitter_id','days_to_death','days_to_last_follow_up','vital_status','TMB']].drop_duplicates()
merged['day_to_endpoint'] = 0
merged

,case_submitter_id,days_to_death,days_to_last_follow_up,vital_status,TMB,day_to_endpoint
0,TCGA-DK-A2I6,'--,2656,Alive,6.835336,0
2,TCGA-FD-A6TK,'--,330,Alive,3.183424,0
4,TCGA-UY-A78L,'--,1127,Alive,5.182940,0
6,TCGA-4Z-AA84,'--,460,Alive,12.011275,0
8,TCGA-XF-A9SK,486,'--,Dead,3.964546,0
...,...,...,...,...,...,...
808,TCGA-FD-A6TA,'--,1912,Alive,7.337379,0
810,TCGA-UY-A8OB,'--,2109,Alive,9.247481,0
812,TCGA-UY-A9PE,'--,189,Alive,5.644348,0
814,TCGA-GU-A42P,332,195,Dead,6.620123,0


# 整合一下死亡及last follow up的数据

In [23]:
merged_T = merged.T
for a,i in merged_T.items():
#     print(a,i['days_to_death']=="'--")
    if i['days_to_death']=="'--":
        i['day_to_endpoint'] = i['days_to_last_follow_up']
    else:
        i['day_to_endpoint'] = i['days_to_death']
merged_T.T.to_csv('./config/TMB&D2D.txt',sep='\t',index)
merged_T.T

,case_submitter_id,days_to_death,days_to_last_follow_up,vital_status,TMB,day_to_endpoint
0,TCGA-DK-A2I6,'--,2656,Alive,6.835336,2656
2,TCGA-FD-A6TK,'--,330,Alive,3.183424,330
4,TCGA-UY-A78L,'--,1127,Alive,5.18294,1127
6,TCGA-4Z-AA84,'--,460,Alive,12.011275,460
8,TCGA-XF-A9SK,486,'--,Dead,3.964546,486
...,...,...,...,...,...,...
808,TCGA-FD-A6TA,'--,1912,Alive,7.337379,1912
810,TCGA-UY-A8OB,'--,2109,Alive,9.247481,2109
812,TCGA-UY-A9PE,'--,189,Alive,5.644348,189
814,TCGA-GU-A42P,332,195,Dead,6.620123,332


In [20]:
os.getcwd()

'/home/wangyh/uro_biomarker/patho_AI'